<a href="https://colab.research.google.com/github/Jeremy26/video_analysis_course/blob/main/Optical_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Tracking & Optical Flow – Intro Workshop
Welcome to your first workshop on Feature Tracking and Optical Flow!<p>

In this workshop, we're going to explore 3 techniques to solve the tracking problem:
*   Feature Tracking
*   Sparse Optical Flow
*   Dense Optical Flow

In the second and third parts, we'll dive into Optical Flow. Both flow estimation techniques can help us track obstacles through time by finding the movement.
![](https://nanonets.com/blog/content/images/2019/04/sparse-vs-dense.gif)

More importantly, we'll try to understand **"What the hell is optical flow?"**


## Imports and Colab

In [ ]:
!wget -qq https://optical-flow-data.s3.eu-west-3.amazonaws.com/images.zip
!unzip -qq images.zip && rm images.zip
!mkdir output
!ls

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from google.colab.patches import cv2_imshow

# 1) Feature Tracking
In this first part, we'll start by tracking visual features.

In [ ]:
#TODO: Load Different Image Pairs
img1 = cv2.imread("images/image0.jpg")
img2 = cv2.imread("images/image1.jpg")

# INITIALIZE FAST DETECTOR AND BRIEF DESCRIPTOR
fast = cv2.xfeatures2d.StarDetector_create()
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

In [ ]:
# DETECTOR
kp = fast.detect(img1,None)

# DESCRIPTOR
kp1, des1 = brief.compute(img1, kp)

# KEYPOINTS DRAWN
img1_kp = cv2.drawKeypoints(img1, kp1, None, color=(0,255,0), flags=0)
cv2_imshow(img1_kp)

In [ ]:
# DETECTOR
kp = fast.detect(img2,None)
# DESCRIPTOR
kp2, des2 = brief.compute(img2, kp)

# KEYPOINTS DRAWN
img2_kp = cv2.drawKeypoints(img2, kp2, None, color=(0,255,0), flags=0)
cv2_imshow(img2_kp)

In [ ]:
#FLANN MATCHING
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(np.float32(des1),np.float32(des2),k=2) # Use NP.FLOAT32 for ORB, BRIEF, etc

# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

if len(good)>10:
    p1 = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    p2 = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                    singlePointColor = None,
                    flags = 2)

img_briefmatch = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
cv2_imshow(img_briefmatch)

# 2) Sparse Optical Flow

In this first part of the tutorial, we'll explore Sparse Optical Flow. This technique is about calculating Optical Flow only for specific features, and not for every pixel. It's **faster** but **not very accurate**.<p> 
The algorithm we'll use for Feature Detection is the **Shi-Tomasi** detector, and the one we'll use for Optical Flow is called **Lucas-Kanade**.<p>
By the way, both algorithms are older than me; so we're really exploring the traditional techniques here 🙃

### First, let's calculate the Sparse Optical Flow on two consecutive images

In [ ]:
#Load two consecutive images
im1 = cv2.imread("images/image0.jpg")
im2 = cv2.imread("images/image1.jpg")

cv2_imshow(im1)
cv2_imshow(im2)

In [ ]:
#Create an Empty mask and define the color of output to green
mask = np.zeros_like(im1)
color = (0, 255, 0)

In [ ]:
#Convert both images to grayscale
gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

cv2_imshow(gray1)
cv2_imshow(gray2)

Define the parameters and launch the **Shi-Tomasi corner detector** on the first image. According to OpenCV: "As usual, image should be a **grayscale image**. Then you specify **number of corners you want to find**. Then you specify the **quality level**, which is a value between 0-1, which denotes the **minimum quality of corner below which everyone is rejected**. Then we provide the **minimum euclidean distance between corners detected**." <p> 👉To understand how to define the parameters, you can use [this link](https://docs.opencv.org/master/dd/d1a/group__imgproc__feature.html#ga1d6bb77486c8f92d79c8793ad995d541).

In [ ]:
feature_params = dict(maxCorners = 300, qualityLevel = 0.0025, minDistance = 15, blockSize = 2) # Default 300, 0.25, 3, 7
prev = cv2.goodFeaturesToTrack(gray1, mask = None, **feature_params)

In [ ]:
print(feature_params)
print(prev[0])

In [ ]:
#Go through each detected corner, and draw a dot
image_corners = im1.copy()

for i in prev:
    x,y = i.ravel()
    cv2.circle(image_corners,(x,y),3,255,-1)
cv2_imshow(image_corners)

Launch the Lucas Kanade Optical Flow algorithm with the features and both grayscale images.<p>
👉 [Here's the link](https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323) to understand the parameters better.

In [ ]:
lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
next, status, error = cv2.calcOpticalFlowPyrLK(gray1, gray2, prev, None, **lk_params)

In [ ]:
print(lk_params)
idx = 19
print(prev[idx])
print(next[idx])
print(status[idx])
print(error[idx])

In [ ]:
#Store the Matches (status=1 means a match)
good_old = prev[status == 1]
good_new = next[status == 1]

In [ ]:
#Go through each matched feature, and draw it on the second image

for new, old in zip(good_new, good_old):
    a, b = new.ravel()
    c, d = old.ravel()
    mask = cv2.arrowedLine(mask, (a, b), (c, d), color, 2)
    im2 = cv2.circle(im2, (a, b), 3, color, -1)
# Overlays the optical flow tracks on the original frame
output = cv2.add(im2, mask)
# Updates previous frame
gray1 = gray2.copy()
# Updates previous good feature points
prev = good_new.reshape(-1, 1, 2)

#cv2.imwrite("output.jpg", output)
cv2_imshow(output)

In [ ]:
cv2_imshow(mask)

### On a video

In [ ]:
def lk_from_image(image):
    global idx, gray1, mask, prev
    if idx==0:
        gray1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        feature_params = dict(maxCorners = 600, qualityLevel = 0.0025, minDistance = 15, blockSize = 2)
        prev = cv2.goodFeaturesToTrack(gray1, mask = None, **feature_params)
        mask = np.zeros_like(image)
        idx+=1
        return image
    # Else
    im2 = image
    if idx%15==0:
        mask = np.zeros_like(image)
    
    if idx%5==0:
        # Every 5 images, relaunch the feature detection
        feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
        prev = cv2.goodFeaturesToTrack(gray1, mask = None, **feature_params)
    
    color = (0, 255, 0)
    gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
 
    lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    next, status, error = cv2.calcOpticalFlowPyrLK(gray1, gray2, prev, None, **lk_params)

    good_old = prev[status == 1]
    good_new = next[status == 1]

    for new, old in zip(good_new, good_old):
        a, b = new.ravel()
        c, d = old.ravel()            
        mask = cv2.arrowedLine(mask, (a, b), (c, d), color, 2)
        im2 = cv2.circle(im2, (a, b), 3, color, -1)
    output = cv2.add(im2, mask)
    gray1 = gray2.copy()
    prev = good_new.reshape(-1, 1, 2)
    idx += 1
    return output

In [ ]:
from moviepy.editor import VideoFileClip
idx = 0

video_file = "images/skateboard.mp4"
clip = VideoFileClip(video_file).subclip(0,20)
white_clip = clip.fl_image(lk_from_image)
%time white_clip.write_videofile("output/output_lk_skateboard.mp4",audio=False)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output/output_lk_skateboard.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# 3) Dense Optical Flow

In this second part, we'll try to learn about Dense Optical Flow; which is used when **we're tracking every pixel of an image**. It's **longer** but **works much better**.

### Let's do the Dense Optical Flow on two consecutive images

In [ ]:
#Load the 2 consecutive images and convert them to grayscale
im1 = cv2.imread("images/image0.jpg")
im2 = cv2.imread("images/image1.jpg")
gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

In [ ]:
#Create a mask in HSV Color Space.
hsv = np.zeros_like(im1)
# Sets image saturation to maximum.
hsv[..., 1] = 255

Now comes the Optical Flow Algorithm. This time, we'll use the Farneback algorithm (2003).<p>
Unlike the Lucas Kanade method, we don't want to find "next, status, and error". This time, we just want to estimate one output, the **flow**.

If you'd like to understand how to tweak the parameters, you can [visit this link](https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback). <p>

👉 The next step is to calculate this flow, and convert it into a visual output.

In [ ]:
flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

### But what is the Flow?
**The Flow is a 2D Matrix** that has the same size as our input frame (image 1). In this matrix, we have the **optical flow vectors U and V**. These are made of point coordinates. **For any point P (x,y) in the grayscale image, the flow contains the corresponding (delta_x, delta_y)** --- how much did that pixel move in the X and Y direction.<p>

In [ ]:
print("Shapes Gray & Flow")
print(gray1.shape)
print(flow.shape)
print(" ")

print("Flow Output")
print(flow)
print(" ")

print("U Vector")
print(flow[...,0])
print(" ")

print("V Vector")
print(flow[...,1])
print(" ")

**We have 2 vectors U and V**, which correspond to the first and second column of the flow output in cartesian coordinates.<p>

---

👉 According to OpenCV: "We get **a 2-channel array** with **optical flow vectors, (u,v)**. We find their **magnitude** and **direction**. We color code the result for better visualization. **Direction corresponds to Hue** value of the image. **Magnitude corresponds to Value plane**."<p>

---

👉 In other words, it's possible to make something with "the point moved 2 pixels to the right"! We find the Magnitude and Angle, convert that to HSV Space (Hue, Saturation, Value) and then make it an image.

I know this sounds horrible, but we need to go back to High School Maths (or maybe college), and remember about Vectors in Cartesian and Polar coordinates. Of course, **the vectors represent the displacements of each pixels.**

*   A vector in **cartesian** coordinates has its values in **(x,y)**.
*   A vector in **polar** coordinates has its values in **Radian**.
![](https://cdn.kastatic.org/ka-perseus-images/1559d8785a298fdd0bac0443388b3812c4327ec3.png)

<p> To compute the Polar coordinates, we must calculate the magnitude and the angle of the vector.

In [ ]:
# Computes the magnitude and angle of the 2D vectors
magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

print(magnitude)
print(angle)

In [ ]:
# Sets image hue according to the optical flow direction
hsv[..., 0] = angle * 180 / np.pi / 2

print(hsv)

In [ ]:
# Sets image value according to the optical flow magnitude (normalized)
hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
print(hsv)

In [ ]:
cv2_imshow(hsv)

In [ ]:
# Converts HSV to RGB (BGR) color representation
rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

In [ ]:
#Display the Output
cv2.imwrite("output/Dense Output.jpg",rgb)
cv2_imshow(rgb)

![](https://www.researchgate.net/profile/Christophoros-Nikou/publication/266149545/figure/fig1/AS:392088710598656@1470492641144/The-optical-flow-field-color-coding-Smaller-vectors-are-lighter-and-color-represents-the.png)

### On a video

In [ ]:
def farneback_from_image(image):
    global idx, im1
    if idx==0:
        im1 = image
        idx+=1
        return im1
    else:
        im2 = image
        gray1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(im1)
        hsv[..., 1] = 255
        flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = angle * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        idx+=1
        im1= image.copy()
        return bgr

In [ ]:
from moviepy.editor import VideoFileClip
idx = 0

video_file = "images/skateboard.mp4"
clip = VideoFileClip(video_file).subclip(0,10)
white_clip = clip.fl_image(farneback_from_image)
%time white_clip.write_videofile("output/output_farneback.mp4",audio=False)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output/output_farneback.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)